<h1><center>Jane Street Market Train Set UMAP Representation with RAPIDS</center></h1>

<center><img src="https://images.exxactcorp.com/CMS/landing-page/resource-center/supported-software/logo/Deep-Learning/Rapids-Logo-lg.png"></center>

In [ ]:
%%time
try:
    import cudf, cuml
    print('rapids already installed')
except:
    # INSTALL RAPIDS OFFLINE (FROM KAGGLE DATASET). TAKES 1 MINUTE :-)
    print('installing rapids (should take ~80sec)')
    import sys
    !cp ../input/rapids/rapids.0.13.0 /opt/conda/envs/rapids.tar.gz  2>/dev/null
    
    !cd /opt/conda/envs/ && tar -xzf rapids.tar.gz 
    sys.path = ["/opt/conda/envs/rapids/lib"] + ["/opt/conda/envs/rapids/lib/python3.6"] + ["/opt/conda/envs/rapids/lib/python3.6/site-packages"] + sys.path
    !cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/  2>/dev/null
    print('done installing rapids')
    import cudf, cuml

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', 500)

import janestreet
import numpy as np
import pandas as pd
from sklearn import preprocessing
from cuml.manifold import UMAP
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")

<a id="1"></a>
<h2 style='background:darkviolet; border:0; color:white'><center>1. Data Loading And Preprocessing<center><h2>

In [ ]:
%%time
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
features = pd.read_csv('../input/jane-street-market-prediction/features.csv')
example_test = pd.read_csv('../input/jane-street-market-prediction/example_test.csv')
sample_prediction_df = pd.read_csv('../input/jane-street-market-prediction/example_sample_submission.csv')
print ("Data is loaded!")

In [ ]:
train = train[train['weight'] != 0]

train['action'] = ((train['weight'].values * train['resp'].values) > 0).astype('int')

X_train = train.loc[:, train.columns.str.contains('feature')]
X_train = X_train.fillna(-999)
y_train = train.loc[:, 'action']

<a id="1"></a>
<h2 style='background:darkviolet; border:0; color:white'><center>2. RAPIDS UMAP Calculation<center><h2>

In [ ]:
umap = UMAP(n_components=2)

embed = umap.fit_transform(X_train.values, y_train.values)[::10, :]

<a id="1"></a>
<h2 style='background:darkviolet; border:0; color:white'><center>3. UMAP components representation with plotly<center><h2>

In [ ]:
colors = {
    0: 'rgba(204, 102, 255, 0.1)',
    1: 'rgba(51, 204, 51, 0.1)'
}

y_train = y_train[::10]
fig = go.Figure(data=[go.Scattergl(
    x=embed[y_train==t, 0],
    y=embed[y_train==t, 1],
    mode='markers',
    marker=dict(
        size=3,
        color=colors[t],
        opacity=0.1
    ),
    name=str(t)
) for t in np.unique(y_train)])

fig.update_layout(title_text='Jane Street Market Train Set UMAP Representation', title_x=0.5, title_y=0.95, template='plotly_dark', margin=dict(l=10, r=10, b=10, t=30))
fig.show();

<a id="1"></a>
<h2 style='background:darkviolet; border:0; color:white'><center>4. References<center><h2>

* [Rapids/UMAP with Fisher metric on RGB histograms](https://www.kaggle.com/hubwag/rapids-umap-with-fisher-metric-on-rgb-histograms)
* [Melanoma tSNE and UMAP embeddings with Rapids](https://www.kaggle.com/tunguz/melanoma-tsne-and-umap-embeddings-with-rapids)